In [35]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages mysql:mysql-connector-java:8.0.24 pyspark-shell"

In [53]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *

In [37]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [63]:
# User schema
schema = StructType([StructField("name", StringType(), True), StructField("age", IntegerType(), True)])

# Read file from hadoop fs
df = spark.read.option("multiline", "true").schema(schema).json("hdfs://hadoop:9000/test/user.json")

df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+------------+---+
|        name|age|
+------------+---+
|Nguyen Van A| 99|
|Nguyen Van B| 66|
|Nguyen Van C| 16|
|Nguyen Van D| 19|
+------------+---+



In [64]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option('driver', 'com.mysql.jdbc.Driver') \
    .option("url", "jdbc:mysql://172.17.0.1:3306/bigdata") \
    .option("dbtable", "user") \
    .option("user", "root") \
    .option("password", "local") \
    .load()

df.write.format('jdbc').options(
          url='jdbc:mysql://172.17.0.1:3306/bigdata',
          driver='com.mysql.jdbc.Driver',
          dbtable='user',
          user='root',
          password='local').mode('append').save()

jdbcDF.show()

+------------+---+
|        name|age|
+------------+---+
|Nguyen Van A| 99|
|Nguyen Van B| 66|
|Nguyen Van C| 16|
|Nguyen Van D| 19|
+------------+---+

